In [ ]:
# Add credentials environment variable here
# Example:
#%env GA4_CREDENTIALS=your/path/to/your/credentials.json

In [2]:
import analytics.charts as ac
import analytics.api as ga
import pandas as pd
from constants import *


In [ ]:
default_params = {
    "service_system": ac.authenticate_ga(SECRET_NAME, ga.ga4_service_params, port=OAUTH_PORT),
}
outlink_search_params = {
    "start_date": '30daysAgo',
    "end_date": TODAY,
    "dimensionFilter": {
        "filter": {
            "stringFilter": {
                "matchType": "EXACT",
                "value": "true"
            },
            "fieldName": "outbound"
        }
    }
}
catalog_params = {
    **default_params,
    **outlink_search_params,
    "property": PROPERTY_ID_MAP[ANVIL_CATALOG_NAME],
}
explorer_params = {
    **default_params,
    **outlink_search_params,
    "property": PROPERTY_ID_MAP[ANVIL_EXPLORER_NAME],
}
portal_params = {
    **default_params,
    **outlink_search_params,
    "base_dimension_filter": EXCLUDE_PAGES_FILTER,
    "property": PROPERTY_ID_MAP[ANVIL_PORTAL_NAME],
}

outlink_params_for_each_property = {
    ANVIL_PORTAL_NAME: portal_params,
    ANVIL_CATALOG_NAME: catalog_params,
    ANVIL_EXPLORER_NAME: explorer_params,
}

In [ ]:
results_dict = {}

for key in outlink_params_for_each_property:
    print(key)
    df_outlinks = (ac.get_data_df(
        ["eventCount"],
        ["pagePath", "linkDomain", "linkUrl"], 
        **outlink_params_for_each_property[key], 
        df_processor=lambda df: df.loc[df.index.get_level_values(1) != ""]
    )).rename_axis(
        index={"pagePath": "Current Page", "linkDomain": "Outlink Domain", "linkUrl": "Outlink URL"}
    )
    results_dict[key] = df_outlinks

with pd.ExcelWriter("outlinks_summary.xlsx") as writer:
    for key in results_dict:
        results_dict[key].to_excel(writer, sheet_name=key)